In [1]:
import os
import pandas as pd
from cleantext import clean
import json
import glob
from pprint import pprint
import datetime
import re  
import nltk  
nltk.download("stopwords")
from nltk.corpus import stopwords
from nltk.stem.porter import *
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from tqdm import tqdm
nltk.download()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Cheng\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [2]:
bitcoin_path = "../Resources/bitcoin.csv"

bitcoin = pd.read_csv(bitcoin_path)

bitcoin.head(50)

,Unnamed: 0,dates,cleaned,negative,neutral,positive,comp
0,0,2021-02-10,"['blue', 'ridg', 'bank', 'share', 'halt', 'nys...",0.000,0.872,0.128,0.2960
1,1,2021-02-10,"['today', 'thursday', 'take', '2', 'friend', '...",0.000,0.775,0.225,0.4939
2,2,2021-02-10,"['guy', 'even', 'read', 'articl', 'btc', 'woul...",0.000,0.719,0.281,0.5719
3,3,2021-02-10,"['btc', 'big', 'chanc', 'billion', 'price', '4...",0.000,1.000,0.000,0.0000
4,4,2021-02-10,"['network', 'secur', '9', '508', 'node', 'toda...",0.200,0.800,0.000,-0.5423
5,5,2021-02-10,"['trade', 'crypto', 'binanc', 'enjoy', 'cashba...",0.000,0.824,0.176,0.4939
6,6,2021-02-10,"['lt', 'fire', 'amp', 'man', 'gt', 'bitcoin', ...",0.178,0.667,0.156,-0.0772
7,7,2021-02-10,"['price', 'updat', 'eur', '1', 'hour', 'btc', ...",0.000,1.000,0.000,0.0000
8,8,2021-02-10,"['btc', 'bitcoin', 'ethereum', 'eth', 'crypto'...",0.000,1.000,0.000,0.0000
9,9,2021-02-10,"['tesla', 'bitcoin', 'invest', 'revolutionari'...",0.000,1.000,0.000,0.0000


In [3]:
df = bitcoin.filter(['text','date'])[0:100000]

df.astype(str).apply(lambda x: x.str.encode('ascii', 'ignore').str.decode('ascii'))


""
0
1
2
3
4
...
99995
99996
99997
99998


In [4]:
def tweet_to_words(tweet):
    text = tweet.lower()
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    words = text.split()
    words = [w for w in words if w not in stopwords.words("english")]
    words = [PorterStemmer().stem(w) for w in words]
    return words

In [5]:
cleantext=[]
for item in tqdm(df['text']):
    words=tweet_to_words(item)
    cleantext+=[words]
df['cleaned']=cleantext
df

KeyError: 'text'

In [ ]:
def unlist(list):
    words=''
    for item in list:
        words+=item+' '
    return words

In [ ]:
def compute_vader_scores(df, label):
    sid = SentimentIntensityAnalyzer()
    df["negative"] = df[label].apply(lambda x: sid.polarity_scores(unlist(x))["neg"])
    df["neutral"] = df[label].apply(lambda x: sid.polarity_scores(unlist(x))["neu"])
    df["positive"] = df[label].apply(lambda x: sid.polarity_scores(unlist(x))["pos"])
    df["comp"] = df[label].apply(lambda x: sid.polarity_scores(unlist(x))["compound"])
    return df

In [ ]:
final_bitcoin = compute_vader_scores(df,'cleaned')
final_bitcoin

In [ ]:
# print(len(date), len(final_bitcoin.cleaned),len(final_bitcoin.negative),len(final_bitcoin.comp),len(final_bitcoin.neutral))

In [ ]:
# final_bitcoin.isnull().values.any()

final_bitcoin[['dates','Time','empty']] = final_bitcoin.date.str.split(" ",expand=True)

final_bitcoin

In [ ]:
final_bitcoin = final_bitcoin.filter(['dates', 'cleaned','negative','neutral','positive','comp'])
final_bitcoin.head(40)

In [ ]:
final_bitcoin.to_csv('bitcoin.csv')
